<img src="images/ProjectPythia_Logo_Final-01-Blue.svg" width=250 alt="Project Pythia Logo"></img>
<img src="images/logos/pangeo_simple_logo.svg" width=250 alt="Pangeo Logo"></img>

# Gulf Stream Currents

---

## Overview

An example that uses `ipyleaflet` to reproduce style of visualization used in the New York Times article
[In the Atlantic Ocean, Subtle Shifts Hint at Dramatic Dangers](https://www.nytimes.com/interactive/2021/03/02/climate/atlantic-ocean-climate-change.html) (March 2, 20121).

1. Open an Intake catalogue reference Sea Surface Height data
2. Make a geographic map of the data using ipyleaflet

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Xarray](https://foundations.projectpythia.org/core/xarray.html) | Helpful |  |
| [Dask](https://docs.dask.org/en/stable/) | Helpful | |
| [Intake](https://intake.readthedocs.io/en/stable/) | Helpful | |
| [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/) | Helpful | |

- **Time to learn**: 15 minutes

## Imports

---

In [1]:
from ipyleaflet import Map, TileLayer, basemaps
from ipyleaflet.velocity import Velocity
from intake import open_catalog

## Load Data

The [Copernicus Monitoring Environment Marine Service (CMEMS)](https://marine.copernicus.eu/) is a large repository of ocean products including in-situ observations, satellite based remote sensing data, and numerical model output.  

We want to look at altimeter satellite data to show the Sea Level Anomalies (SLA) for the global ocean. The particular data product is called **Global Ocean Gridded L4 Sea Surface Heights and Derived Variables Reprocessed (1993-Ongoing)** ([SEALEVEL_GLO_PHY_L4_MY_008_047](https://resources.marine.copernicus.eu/product-detail/SEALEVEL_GLO_PHY_L4_MY_008_047/INFORMATION)).

This dataset is available as an analysis-ready on the Pangeo Cloud Data Catalog 


In [2]:
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean.yaml")
cat["sea_surface_height"]

sea_surface_height:
  args:
    consolidated: true
    storage_options:
      requester_pays: true
    urlpath: gs://pangeo-cmems-duacs
  description: sea-surface altimetry data from The Copernicus Marine Environment
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs
    tags:
    - ocean
    - satellite
    url: http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047


This dataset is marked "requester pays" which means we have do an addtional step if we are not already on Pangeo Hub on the Google Cloud Platform.

### Working with requester pays data
Several of the datasets within the Pangeo cloud data catalog are contained in [requester pays](https://cloud.google.com/storage/docs/requester-pays) storage buckets. This means that a user requesting data must provide their own billing project (created and authenticated through Google Cloud Platform) to be billed for the charges associated with accessing a dataset. To set up an GCP billing project and use it for authentication in applications:
- [Create a project on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects#creating_a_project); if this is the first time using GCP, a prompt will appear to choose a Google account to link to all GCP-related activities.
- [Create a Cloud Billing account](https://cloud.google.com/billing/docs/how-to/manage-billing-account#create_a_new_billing_account) associated with the project and [enable billing for the project](https://cloud.google.com/billing/docs/how-to/modify-project#enable_billing_for_a_project) through this account.
- Using [Google Cloud IAM](https://cloud.google.com/iam/docs/granting-changing-revoking-access#granting-console), add the **Service Usage Consumer** role to your account, which enables it to make billed requests on the behalf of the project.
Through command line, install the [Google Cloud SDK](https://cloud.google.com/sdk); this can be done using conda:
 
    ```conda install -c conda-forge google-cloud-sdk```

- Initialize the `gcloud` command line interface, logging into the account used to create the aforementioned project and selecting it as the default project; this will allow the project to be used for requester pays access through the command line:

    ```gcloud auth login
gcloud init```
- Finally, use `gcloud` to establish application default credentials; this will allow the project to be used for requester pays access through applications:

   ```gcloud auth application-default login```

In [3]:
ds  = cat["sea_surface_height"].to_dask()
ds

<xarray.Dataset>
Dimensions:    (time: 8901, latitude: 720, longitude: 1440, nv: 2)
Coordinates:
    crs        int32 ...
    lat_bnds   (time, latitude, nv) float32 dask.array<chunksize=(5, 720, 2), meta=np.ndarray>
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
    lon_bnds   (longitude, nv) float32 dask.array<chunksize=(1440, 2), meta=np.ndarray>
  * longitude  (longitude) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * nv         (nv) int32 0 1
  * time       (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2017-05-15
Data variables:
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(5, 720, 1440), meta=np.ndarray>
    err        (time, latitude, longitude) float64 dask.array<chunksize=(5, 720, 1440), meta=np.ndarray>
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(5, 720, 1440), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 dask.array<chunksize=(5, 720, 1440), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 dask.array<chunksize=(5, 720, 1440), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 dask.array<chunksize=(5, 720, 1440), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 dask.array<chunksize=(5, 720, 1440), meta=np.ndarray>
Attributes: (12/43)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-4 sea su...
    time_coverage_duration:          P1D
    time_coverage_end:               1993-01-01T12:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             1992-12-31T12:00:00Z
    title:                           DT merged all satellites Global Ocean Gr...

## Make a Map

In [4]:
center = [35, -50]
zoom = 4
m = Map(center=center, zoom=zoom, interpolation='nearest', basemap=basemaps.Gaode.Satellite)

display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}

wind = Velocity(
    data=ds.isel(time=-1), 
    zonal_speed='ugos', meridional_speed='vgos', 
    latitude_dimension='latitude', longitude_dimension='longitude', 
    velocity_scale=0.2, max_velocity=1, 
    display_options=display_options
)

m.add_layer(wind)

m

Map(center=[35, -50], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

---

## Summary

In this example we loaded sea level data from an analysis-ready cloud based dataset and made a visualization of that data using mapping library.

## Resources and references
- This notebook is based on the Pangeo physical oceanography gallery example: <https://gallery.pangeo.io/repos/pangeo-gallery/physical-oceanography/05_gulf_stream_currents.html>